Note on Singleton Pattern

Sometimes you need to create object, which will serve 
as "storage" for some variables, through instance attributes.
For example, you need to cache some data.

But the thing is, that you should have only one instance of that class
throughout your prorgramm or separate scripts.
So, if instance of some class have been already created before, 
any attempts of creating another object should do nothing.
Really, imagine you have two or more different objects that contains same type
of data and serving the same purpose? How to control over that?

How do you think, is it possible to do so in Java?

Let's break down the question into several parts.
Suppose I have a class which serves the cache purposes:

In [21]:
class Cache():

    _cache = {}

    def __init__(self):
        pass
    
    @staticmethod  # don't worry about this @ wrapper, it's not the point here
    def get_cache(key):
        return Cache._cache[key]
    
    def add_cache(self, key, val):
        Cache._cache[key] = val
        return self

In [22]:
cache = Cache()

In [23]:
cache.add_cache('place', 'potapsco')

In [24]:
Cache.get_cache('place')

'potapsco'

Nice, it works. But I can create several Cache instances, which can mess up everything.

Also, name cache isn't available now. How to avoid multiple instance and store everything in one class?
I should mention that it's not only about cache, e.g. you can control over number of connections, requests and other stuff. There are a lot of examples where you need only one class instance! You might need to store some settings which should be stored only once and be accessible from any place.

Yes, we can use some flags, global variables, which is indeed bad practice (almost any global variable is bad practice). But there is "singleton pattern" which is elegant and concise way to solve this problem.

It's power depends on the core of Python ideas, which unite both object-oriented and functional programming under same roof.

What happens when I do cache = Cache()? How class is actually created?

It seems that __ init __  which I used in Cache class, prompted first and *this* method creates an object. But actually it only **initializes** the attributes of already created class instance. 

Behind the scenes, there is more deep, __ new __ method, which **creates** object itself, without any attributes and methods, just allocating some place for object in memory. This method executed by default firstly any time you call (create) any object. You can't control over memory it uses like in C++, but *you can control* the process of the object "construction".

And the main trick is, that you can use simple if statement which will create object if any other exist, otherwise - just do nothing. What a power of Python!

Let me show you that:

In [25]:
class Cache():

    _instance = None # class attribute which shared across any current class objects. Simple flag

    def __new__(cls, *args, **kwargs): # Here we go! We are using the method right from the Python's heart
        if cls._instance is None:
            cls._instance = super(Cache, cls).__new__(cls)
            # This row is more than tricky at the first sight, but let me explain that.
            # super() returns the class instance of parent class, in this case, parent of Cache class.
            # But there is no any other classes except only Cache, isn't it?
            # The thing is, that any python class always inherits core, superparent class "object"
            # Which has another from here defined method __new__ (at it's default shape)
            # The purpose of super(Cache, cls) is to return parent class which is able to create not only one
            # but any number of objects! 
            # But if no parent class exists, "object" is called, which is very basic built-in core class instance.
            # Thus, super(Cache, cls).__new__(cls) will create an object itself anyway :)
            # After that, cls._instance will not be None any time, and "constructor" will return the very first version
            # of object (when first time called) it created. 
            # This is what is called "Singleton pattern"
        return cls._instance

    # Other stuff remains untouched
    def __init__(self):
        pass
    
    @staticmethod
    def get_cache(key):
        return Cache._cache[key]
    
    def add_cache(self, key, val):
        Cache._cache[key] = val
        return self

How can i use that in practice?

Let's say, you have a function or its part, which stores some data into Cache:

In [26]:
def jazz_func(key, val):
    cache = Cache._instance 
    # No worries, Cache._instance, if exists, could be only one!
    # Also, we can avoid even using cache name, just by using Cache._instance any time
    if cache.get_cache('user1234') is None: # check if user exists
        cache.add_cache('user1234', 'password1234') # add if not
        return "User added"
    else:
        return "User with this name already exists"

Or you might have function which should on first call calculate something and store that, but on next calls use already stored parameters:

In [27]:
class Borscht():

    _instance = None

    def __new__(cls, *args, **kwargs):
        if cls._instance is None:
            cls._instance = super(Borscht, cls).__new__(cls)
        return cls._instance

    def __init__(self, meat, water):
        if not hasattr(self, "bouillon_ready"):
            self.meat = meat
            self.water = water
            self._bouillon = f"Borscht setup: type of meat - {meat}, water - {water}"
            self.bouillon_ready = True  # make sure bouillon is cooked

    def get_bouillon(self):
        return self._bouillon

In [28]:
def cook_borscht(meat, water, vegetables):
    if Borscht._instance is None:
        base = Borscht(meat, water) # you can avoid assigning base, just by using Borscht._instance every time
        print("Bouillon is ready and stored")
        print(f"Ingridients used: meat - {base.meat}, water - {base.water}")
    else:
        # Access the singleton instance to get the bouillon
        base = Borscht._instance
        bouillon = base.get_bouillon()
        print("Using precooked bouillon")
        print(f"{bouillon}, {vegetables}")

In [29]:
cook_borscht(meat="beef", water="regular", vegetables="beets, carrots")

Bouillon is ready and stored
Ingridients used: meat - beef, water - regular


In [30]:
cook_borscht(meat="pork", water="uncommon", vegetables="beets, carrots")

Using precooked bouillon
Borscht setup: type of meat - beef, water - regular, beets, carrots


See, no pork is here, and uncommon water as well. Because once object is created, no way other objects of this class could be created, even by mistake.